<a href="https://colab.research.google.com/github/limshaocong/analyticsEdge/blob/main/Zero_shot_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==3.1.0

     |████████████████████████████████| 884 kB 8.4 MB/s 
     |████████████████████████████████| 3.0 MB 34.2 MB/s 
     |████████████████████████████████| 895 kB 63.1 MB/s 
     |████████████████████████████████| 1.2 MB 55.9 MB/s 


In [ ]:
from transformers import pipeline
import pandas as pd

In [ ]:
classifier = pipeline("zero-shot-classification", # transfer learning
                      model = "joeddav/xlm-roberta-large-xnli", # multi-lingual model
                      device = 0) # to utilize GPU

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [ ]:
import re
from datetime import datetime
import os.path
from os import path
import requests

In [ ]:
def analyse(ticker):
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  df = pd.read_csv(file_path)
  
  df['Text_clean'] = df['Text_clean'].astype(str)

  texts = df["Text_clean"]

  chunks = [texts[x : x + 100] for x in range(0, len(texts), 100)] # split into chunks

  results_list = []

  for idx in range(len(chunks)):

    cp1 = datetime.now()
    
    chunk = chunks[idx].tolist()

    candidate_labels = ["positive", "negative", "neutral"]

    hypothesis_template = "The sentiment of this tweet is {}."

    #dicts = classifier(text, candidate_labels, hypothesis_template = hypothesis_template)
    #result = dicts["labels"][0]
    #results_list.append(result)
    
    list_results_dicts = classifier(chunk, candidate_labels, hypothesis_template = hypothesis_template)

    if type(list_results_dicts) != dict:
      new_list = [i["labels"][0] for i in list_results_dicts] # not robust when chunk size == 1
      results_list.extend(new_list)
    
    else:
      new_list = list_results_dicts["labels"][0]
      results_list.append(new_list)
    
    #results_list.extend(new_list)

    cp2 = datetime.now()

    print(str(idx + 1) + str("/") + str(len(chunks)) + str(" --- ") + str(cp2 - cp1))

  return results_list

In [73]:
from google.colab import files

ticker_path = "https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/Tickers_reduced_full.csv"
tickers = pd.read_csv(ticker_path)
tickers_list = tickers["Tickers"].tolist()
tickers_list = tickers_list[70:]

path = "https://raw.githubusercontent.com/limshaocong/analyticsEdge/main//Datasets/Tweet_Analysis/"

In [74]:
tickers_list[:5]

['INTC', 'JNJ', 'JPM', 'JW.A', 'KALU']

In [68]:
skipped = ['AMZN',
 'BA',
 'BRK.B',
 'BSX',
 'C',
 'CAT',
 'CENT',
 'CENTA',
 'CHK',
 'CMCSA',
 'COR',
 'CORE',
 'CPS',
 'CRDA',
 'CRM',
 'CSCO',
 'CSR',
 'CUBI',
 'DISCA',
 'DISCK',
 'DOV',
 'DUK',
 'EBAY',
 'EL',
 'EPR',
 'F',
 'FB',
 'FCNCA',
 'FNB',
 'FOX',
 'FOXA',
 'FWONA',
 'FWONK', 'GOOG', 'GOOGL', 'IBM']

In [ ]:
for ticker in tickers_list:
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  response = requests.get(file_path)

  if response.status_code < 400:
 
    print(ticker + str(" is present"))

    file_path = path + ticker + str("_tweets_2020_cleaned.csv")

    print(file_path)
  
    df = pd.read_csv(file_path)

    results_list = analyse(ticker)

    df["Sentiment"] = results_list

    newfilename = ticker + str("_tweets_2020_tagged.csv")

    df.to_csv(newfilename, index = False)
  
  else:

    print(ticker + str(" is missing"))

    skipped.append(ticker)

    print(skipped)

    continue